In [ ]:
%matplotlib inline


# True positive proportions in fMRI clusters using Notip

This script showcases the so-called Notip procedure [1], in
which the proportion of true discoveries in arbitrary clusters is estimated.
The clusters can be defined from the input image, i.e. in a circular way, as
the error control accounts for arbitrary cluster selection.


In [ ]:
pip install notip -q

## Fetch dataset
We download a list of left vs right button press contrast maps from the so-called localizer dataset [2]. Note that we fetch individual t-maps that represent the
BOLD activity estimate divided by the uncertainty about this estimate.

In [ ]:
from nilearn.datasets import fetch_localizer_contrasts
import numpy as np

n_subjects = 30
data = fetch_localizer_contrasts(
    ["left vs right button press"],
    n_subjects,
    get_tmaps=True,
    legacy_format=False,
)
# TODO: explain the meaning of the contrast used
# TODO: explain that we take t maps

Let's visualize the input data:

In [ ]:
import matplotlib.pyplot as plt
from nilearn.plotting import plot_glass_brain
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(6, 6), dpi=300)
# we only look at 16 subjects
for cidx, tmap in enumerate(data['tmaps'][:16]):
    plot_glass_brain(
        tmap,
        colorbar=False,
        threshold=2.0,
        axes=axes[int(cidx / 4), int(cidx % 4)],
        plot_abs=False,
        annotate=False,
        display_mode='z')

To extract signal from these images, we need a masker:

In [ ]:
from nilearn.maskers import NiftiMasker
smoothing_fwhm = 8.0
nifti_masker = NiftiMasker(smoothing_fwhm=smoothing_fwhm)

In [ ]:
fmri_input = nifti_masker.fit_transform(data["cmaps"])
# If we use that, no need to download / show the tmaps

In [ ]:
print(fmri_input.shape)

We have extracted the values of 46482 voxels from 30 images.

### Computing True Discovery Proportion (TDP) lower bounds on data-derived clusters

First, we need to compute a statistical map from the input data. This is done via a t-test, performed for each voxel.

In [ ]:
from scipy import stats
from scipy.stats import norm

# Let's run a one-sample t test on these data
stats_, p_values = stats.ttest_1samp(fmri_input, 0)
# Let's z-transform these p-values into z values
z_vals_ = norm.isf(p_values)
# Let's make this an image by using the inverse_transform method of the masker
z_map = nifti_masker.inverse_transform(z_vals_)
# todo :immediately plot the z_map

We can now use Notip to compute TDP lower bounds on clusters exceeding a z-value threshold. Note that since we study a contrast that corresponds to the difference of two conditions, we can only perform one-sample testing.

In [ ]:
from notip.posthoc_fmri import get_clusters_table_with_TDP
get_clusters_table_with_TDP(z_map, fmri_input, n_permutations=200, stat_threshold=3.5, methods=['Notip'])
# todo: rename to get_clusters_table_TDP_1samp
# todo: create get_clusters_table_TDP_2samp
# todo: remove z_map

We have reduced the number of permutations to 200 for the sake of computation time. Note that we can get tighter FDP control by increasing this number.

### Comparison with other TDP lower bounds 

There exist other approach to get TDP estimates. Among those, All-Resolution-Inference (ARI) does not require permutations. Let us compare their result.

In [ ]:
from notip.posthoc_fmri import get_clusters_table_with_TDP
get_clusters_table_with_TDP(
    z_map, 
    fmri_input, 
    n_permutations=200,
    stat_threshold=3.5,
    methods=['ARI', 'Notip'])

### Using Notip on anatomical regions from atlases

In [ ]:
from nilearn import datasets
atlas = datasets.fetch_atlas_harvard_oxford('cort-prob-1mm')
# todo: take 2mm atlas for the sake of computation time
atlas_filename = atlas.maps
labels = atlas.labels[1:]
atlas_masked = nifti_masker.transform(atlas_filename)

In [ ]:
len(labels)

We have 48 atlas regions.

In [ ]:
import numpy as np
from nilearn._utils.niimg import safe_get_data
# idx = np.random.randint(len(labels)) # Choose an atlas region
idx = 6
# FIXME: why aren't you using a proper masker ? the manipulation below is horrible
region_mask_ = np.where(safe_get_data(nifti_masker.inverse_transform(atlas_masked))[:,:,:,idx] != 0)
region_mask = np.zeros(z_map.shape)
region_mask[region_mask_] = 1
#Awful hack, use math_img

In [ ]:
labels[idx]

In [ ]:
np.count_nonzero(region_mask)

We choose the Precentral Gyrus, comprising 7051 voxels.

In [ ]:
from notip.posthoc_fmri import get_tdp_bound_notip
notip_bound, cluster_map = get_tdp_bound_notip(z_map, fmri_input, region_mask, n_permutations=200)
# rename tdp_bound_notip_1samp()
# remove z_map

In [ ]:
from nilearn.plotting import plot_stat_map
plot_stat_map(cluster_map, title='TDP > %s' % ("{0:.2f}".format(notip_bound)))

### Using Notip on user-defined clusters

We will now use Notip on clusters extracted from the data; we seek to find connected components exceeding a z-value threshold. We set the threshold to 3.5.

In [ ]:
stat_threshold = 3.5
# should have been defined earlier, once and for all.

In [ ]:
import numpy as np
from scipy import ndimage
from nilearn._utils.niimg import safe_get_data


# Defining "faces" connectivity for voxels

conn_mat = np.zeros((3, 3, 3), int)
conn_mat[1, 1, :] = 1
conn_mat[1, :, 1] = 1
conn_mat[:, 1, 1] = 1

stat_map = safe_get_data(z_map, ensure_finite=True)

# Thresholding the map
binarized = stat_map > stat_threshold
binarized = binarized.astype(int)

# Extracting clusters from thresholded map
label_map = ndimage.measurements.label(binarized, conn_mat)[0]

cluster_mask = label_map == 1 # First cluster

# There are functions in nilearn that do that.
# 

In [ ]:
np.count_nonzero(cluster_mask)

This cluster comprises 796 voxels.

In [ ]:
from notip.posthoc_fmri import get_tdp_bound_notip
notip_bound, cluster_map = get_tdp_bound_notip(z_map, fmri_input, cluster_mask, n_permutations=200)
# todo: tdp_bound_notip_1samp

Let's visualize the results:

In [ ]:
from nilearn.plotting import plot_stat_map
plot_stat_map(cluster_map, title='TDP > %s' % ("{0:.2f}".format(notip_bound)))

In [ ]:
# todo: use BH threshold to illustrate that FDR control does not yeld FDP control

# todo: add an example to play with alpha

# References

Blain, Alexandre, Bertrand Thirion, and Pierre Neuvial. "Notip: Non-parametric True Discovery Proportion control for brain imaging." NeuroImage 260 (2022): 119492. doi:https://doi.org/10.1016/j.neuroimage.2022.119492

Dimitri Papadopoulos Orfanos, Vincent Michel, Yannick Schwartz, Philippe Pinel, Antonio Moreno, Denis Le Bihan, and Vincent Frouin. The brainomics/localizer database. NeuroImage, 144:309–314, 2017. Data Sharing Part II. URL: https://www.sciencedirect.com/science/article/pii/S1053811915008745, doi:https://doi.org/10.1016/j.neuroimage.2015.09.052.